# Introduction 
This Jupyter Notebook is a walkthrough of the exploration and preparation that went into the Shanghai Smog Visualization.

Notebook is written in Python 3.6     
Required Packages: Pandas

In [1]:
# Import necessary packages for data parsing and exploration
import pandas as pd

## Gather

In [2]:
# Import CSV files from downloaded sources. 
# Online source is at http://www.stateair.net/web/historical/1/4.html Accessed: Febuary 18, 2018
# The United States Department of State website on Shanghai Historical Data on Air Quality

S2011 = pd.read_csv('Shanghai_2011_HourlyPM25_created20140423.csv', skiprows = 3, encoding = 'windows-1252')
S2012 = pd.read_csv('Shanghai_2012_HourlyPM25_created20140423.csv', skiprows = 3, encoding = 'windows-1252')
S2013 = pd.read_csv('Shanghai_2013_HourlyPM25_created20140423.csv', skiprows = 3, encoding = 'windows-1252')
S2014 = pd.read_csv('Shanghai_2014_HourlyPM25_created20150203.csv', skiprows = 3, encoding = 'windows-1252')
S2015 = pd.read_csv('Shanghai_2015_HourlyPM25_created20160201.csv', skiprows = 3, encoding = 'windows-1252')
S2016 = pd.read_csv('Shanghai_2016_HourlyPM25_created20170201.csv', skiprows = 3, encoding = 'windows-1252')
S2017 = pd.read_csv('Shanghai_2017_HourlyPM25_created20170803.csv', skiprows = 3, encoding = 'windows-1252')

### Define - Combine Datasets
Right now all the datasets are separate. I would like to combine them into a single dataset to prepare for cleaning.      
From a separate view, I am 100% sure that the formats are identical and they can be concatenated without an issue.
### Code

In [3]:
# Create a list of each dataframe
list = [S2011, S2012, S2013, S2014, S2015, S2016, S2017]

# Create an empty dataframe aggregator variable
agg = pd.DataFrame()

# Using a for loop, aggregate all dataframes into one list
for df in list:
    agg = pd.concat([agg, df])

### Test
The describe method on the Year column shows that I have successfully concatenated the results. The counts are much higher, and the span of the Years included in the aggregated dataframe includes the years form 2011 to 2017.

In [4]:
S2011.Year.describe()

count    8760.0
mean     2011.0
std         0.0
min      2011.0
25%      2011.0
50%      2011.0
75%      2011.0
max      2011.0
Name: Year, dtype: float64

In [5]:
agg.Year.describe()

count    56952.000000
mean      2013.767383
std          1.886334
min       2011.000000
25%       2012.000000
50%       2014.000000
75%       2015.000000
max       2017.000000
Name: Year, dtype: float64

## Assess

In [6]:
agg.head()

,Site,Parameter,Date (LST),Year,Month,Day,Hour,Value,Unit,Duration,QC Name
0,Shanghai,PM2.5,2011-01-01 00:00,2011,1,1,0,-999,µg/m³,1 Hr,Missing
1,Shanghai,PM2.5,2011-01-01 01:00,2011,1,1,1,-999,µg/m³,1 Hr,Missing
2,Shanghai,PM2.5,2011-01-01 02:00,2011,1,1,2,-999,µg/m³,1 Hr,Missing
3,Shanghai,PM2.5,2011-01-01 03:00,2011,1,1,3,-999,µg/m³,1 Hr,Missing
4,Shanghai,PM2.5,2011-01-01 04:00,2011,1,1,4,-999,µg/m³,1 Hr,Missing


In [7]:
col = [0, 1, 8, 9, 10]
for elem in col:
    print(agg.iloc[:, elem].value_counts())

Shanghai    56952
Name: Site, dtype: int64
PM2.5    56952
Name: Parameter, dtype: int64
µg/m³    56952
Name: Unit, dtype: int64
1 Hr    56952
Name: Duration, dtype: int64
Valid      47324
Missing     9628
Name: QC Name, dtype: int64


A quick look at the dataset using the head function shows that we have several paramaters that we can remove. 
- Site is consistent
- Parameter is consistent (particle size)
- Date already exists in other values, it can be removed
- Value has -999 for non recorded data points. Need to be removed
- Units can be added later and are not valuable in this firm
- Duration is the same 
- QC Name is for missing and non missing data. Can be removed

### Define - (-)999
I will remove the filled null entries to prevent the skewing of data in the visualizations. 
### Code

In [8]:
# Create copy to preseve original data
df = agg.copy()

# Remove null entries
df = df[df.Value != -999]

### Test

In [9]:
df.head()

,Site,Parameter,Date (LST),Year,Month,Day,Hour,Value,Unit,Duration,QC Name
8682,Shanghai,PM2.5,2011-12-28 18:00,2011,12,28,18,36,µg/m³,1 Hr,Valid
8683,Shanghai,PM2.5,2011-12-28 19:00,2011,12,28,19,41,µg/m³,1 Hr,Valid
8684,Shanghai,PM2.5,2011-12-28 20:00,2011,12,28,20,44,µg/m³,1 Hr,Valid
8685,Shanghai,PM2.5,2011-12-28 21:00,2011,12,28,21,40,µg/m³,1 Hr,Valid
8686,Shanghai,PM2.5,2011-12-28 22:00,2011,12,28,22,25,µg/m³,1 Hr,Valid


In [10]:
df.Year.value_counts()

2014    8657
2013    8587
2016    8546
2012    8505
2015    8385
2017    4271
2011      77
Name: Year, dtype: int64

There are some missing data, this should be taken into consideration.     
I will omit 2011 and 2017 because of how much missing data there are, and I expect some trends to be seasonal and be auto regressive. 

### Define - Column Cleaning
Remove uncessary columns to shrink down the dataset
### Code

In [11]:
# Get only useful columns
df['Date'] = df['Date (LST)']
df = df[['Date', 'Value']]

### Test

In [12]:
df.head()

,Date,Value
8682,2011-12-28 18:00,36
8683,2011-12-28 19:00,41
8684,2011-12-28 20:00,44
8685,2011-12-28 21:00,40
8686,2011-12-28 22:00,25


In [13]:
# Save as csv for easier processing in visualizers
df.to_csv('Shanghai_Smog_Data.csv', encoding = 'utf-16')

# Conclusion
In this notebook I explored and prepared the Shanghai Smog dataset for visualization in Tableau by removing unnecessary columns and empty rows. 